In [1]:
import os
from typing import List, Dict
from tqdm import tqdm

import numpy as np
import pandas as pd

# for molecular similarity
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem.Scaffolds.MurckoScaffold import MakeScaffoldGeneric as GraphFramework
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol
from Levenshtein import distance as levenshtein

# for `def random_split`
import random
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
# for `def cluster_split` or `def diverse_split`
from rdkit.ML.Cluster import Butina 
# for `def cluster_split_old``
from sklearn.cluster import SpectralClustering 

from datacat4ml.const import RANDOM_SEED
from datacat4ml.utils import mkdirs
from datacat4ml.const import CURA_LHD_OR_DIR, CURA_MHD_OR_DIR, CURA_MHD_OR_effect_DIR, CURA_HHD_OR_DIR
from datacat4ml.const import SPLIT_LHD_OR_DIR, SPLIT_MHD_OR_DIR, SPLIT_MHD_OR_effect_DIR, SPLIT_HHD_OR_DIR

# Functions

In [ ]:
#===================1. Molecular distance  ====================
#  Substructure distance based on morgan fingerprint
def get_substructure_dmat(smiles: List[str], radius: int = 2, nBits: int = 1024):

    """ 
    Calculates a matrix of Tanimoto distance scores for the whole molecules of a list of SMILES string.
    
    This method capture the “global” differences between molecules by considering the entire set of substructures they contain
    """

    fps = []
    for smi in smiles:
        mol = Chem.MolFromSmiles(smi)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
        fps.append(fp)
    
    # generate the distance matrix based on the fingerprints:
    dmat = np.zeros((len(fps), len(fps)), float)
    for i, fp in enumerate(fps):
        if i == len(fps) - 1:
            break
        ds = np.array(
            DataStructs.BulkTanimotoSimilarity(fp,
                                               fps[i + 1:],
                                               returnDistance=1)) # Distance = 1 - Similarity. Set returnDistance=1 to get distance, 0 for similarity.
        dmat[i, i + 1:] = ds
        dmat[i + 1:, i] = ds
    
    return dmat
    
# scaffold distance based on morgan fingerprint
def get_scaffold_dmat(smiles: List[str], radius: int = 2, nBits: int = 1024):
    """ Calculates a matrix of Tanimoto distance scores for the scaffolds of a list of SMILES string """
    scaf_fps = {}
    for smi in smiles:
        mol = Chem.MolFromSmiles(smi)
        try:
            skeleton = GraphFramework(mol) # returns the generic scaffold graph, whcih represents the connectivity and topology of the molecule
        except Exception: # In the very rare case that the molecule cannot be processed, then use a normal scaffold
            print(f"Could not create a generic scaffold of {smi}, then used a normal scaffold instead")
            skeleton = GetScaffoldForMol(mol) # returns the Murcko scaffold, which is the result of removing side chains while retaining ring systems
        skeleton_fp = AllChem.GetMorganFingerprintAsBitVect(skeleton, radius=radius, nBits=nBits)
        scaf_fps.append(skeleton_fp)

    dmat = np.zeros((len(smiles), len(smiles)), float)
    for i, scaf_fp in enumerate(scaf_fps):
        if i == len(scaf_fps) - 1:
            break
        ds = np.array(
            DataStructs.BulkTanimotoSimilarity(scaf_fp,
                                               scaf_fps[i + 1:],
                                               returnDistance=1)) # Distance = 1 - Similarity. Set returnDistance=1 to get distance, 0 for similarity.
        dmat[i, i + 1:] = ds
        dmat[i + 1:, i] = ds

    return dmat

# levenstein distance based on SMILES strings
def get_levenshtein_dmat(smiles: List[str], normalize: bool = True):
    """ Calculates a matrix of levenshtein similarity scores for a list of SMILES string
    Levenshtein similarity, i.e edit distance similarity, measures the number of single character edits (insertions, deletions or substitutions) required to change one string into the other.
    Ad SMILES is a text-based representation of a molecule, this similarity metric can be used to measure the similarity between two molecules.
    
    """
    smi_len = len(smiles)

    matrix = np.zeros([smi_len, smi_len])
    # calcultate the upper triangle of the matrix
    for i in tqdm(range(smi_len)):
        for j in range(i, smi_len):
            if normalize:
                matrix[i,j] = levenshtein(smiles[i], smiles[j]) / max(len(smiles[i]), len(smiles[j])) # calculate the distance and normalize it to [0,1]
            else:
                matrix[i,j] = levenshtein(smiles[i], smiles[j]) # calculate the distance without normalization
    # fill in the lower triangle without having to loop (saves ~50% of time)
    matrix = matrix + matrix.T - np.diag(np.diag(matrix))
    # get from a distance matrix to a similarity matrix
    matrix = 1 - matrix

    # fill the diagonal with 0's
    np.fill_diagonal(matrix, 0)

    return matrix

def molecule_similarity(smiles: List[str], similarity: float = 0.9,):
    """ Calculate which pairs of molecules have a high substructure, scaffold, or SMILES similarity """
    m_subs = get_substructure_dmat(smiles) <= (1 - similarity)
    m_scaff = get_scaffold_dmat(smiles) <= (1 - similarity)
    m_leve = get_levenshtein_dmat(smiles) <= (1 - similarity)

    return (m_subs + m_scaff + m_leve).astype(int)

def find_stereochemical_siblings(smiles: List[str]):
    """
    """
    lower = np.tril(get_substructure_dmat(smiles, radius=4, nBits=4096), k=0)
    identical = np.where(lower == 1) # identical[0] is the row indices, identical[1] is the column indices
    identical_pairs = [[smiles[identical[0][i]], smiles[identical[1][i]]] for i, j in enumerate(identical[0])] # a list of 2-element lists, each containing a pair of identical SMILES strings.
                       
    return list(set(sum(identical_pairs, []))) # a list of unique SMILES strings that have at least one stereochemical sibling in the input list `smiles`.

In [94]:
# random split
def random_split(x, y, n_folds=5, test_size=0.2, random_seed=RANDOM_SEED):
    """
    randomly split the dataset into training and testing sets for n_folds times, stratified on y

    returns
    -----------
    - test_splits: List[np.ndarray]
        A list of lists, each containing the indices of the test set for each fold.
    """
    sss = StratifiedShuffleSplit(n_splits=n_folds, test_size=test_size, random_state=random_seed)
    test_folds = []
    for train_idx, test_idx in sss.split(x, y):
        test_folds.append((test_idx.tolist()))
    return test_folds

def random_split_old(smiles: List[str], activity: List[float], activity_id: List[str], 
                 random_seed: int = RANDOM_SEED,
                 n_folds: int=5, test_size: float=0.2):
    """
    
    returns
    -----------
    - train_act_id: Dict[str: List[str]]
    - test_act_id: Dict[str: List[str]]
        Dicts of activity IDs for the training and testing sets of each fold, respectively.
        
    """
    sss = StratifiedShuffleSplit(n_splits=n_folds, test_size=test_size, random_state=random_seed)

    fold_train_act_id, fold_test_act_id = {}, {}
    #train_act_id, test_act_id = [], []
    # check if the y contains at least 2 classes
    unique_classes = set(activity)
    if len(unique_classes) < 2:
        raise ValueError("The target variable 'activity' must contain at least two classes for stratified splitting.")
    else:
        for i, (train_index, test_index) in enumerate(sss.split(smiles, activity)):# stratified on y, i.e, `activity`
            #print(f'folder {i}:')
            #print(f'Train_length = {len(train_index)}\nTrain: index = {train_index}, ')
            #print(f'Test_length = {len(test_index)}\nTest: index = {test_index}')
            train_act_id = [activity_id[i] for i in train_index]
            test_act_id = [activity_id[i] for i in test_index]

            fold_train_act_id[f'fold_{i}'] = train_act_id
            fold_test_act_id[f'fold_{i}'] = test_act_id
            #print('----------------------------------')

        return fold_train_act_id, fold_test_act_id

# cluster split_old
"""Adopted from MoleculeACE"""
def cluster_split_old(df: pd.DataFrame, aim: str = 'lo', 
                  n_clusters: int=5, test_size: float=0.2, remove_stereo: bool = True):
    """
    
    returns
    -----------
    - train_act_id: List[str]
    - test_act_id: List[str]
        Lists of activity IDs for the training and testing sets, respectively.
    """
    smiles = df['canonical_smiles_by_Std'].tolist()
    if aim == 'vs':
        activity = df['vs_activity'].tolist()
    elif aim == 'lo':
        activity = df['lo_activity'].tolist()
    activity_id = df['activity_id'].tolist()

    if remove_stereo:
        stereo_smiles_idx = [smiles.index(i) for i in find_stereochemical_siblings(smiles)]
        smiles = [smi for i, smi in enumerate[smiles] if i not in stereo_smiles_idx]
        activity = [act for i, act in enumerate[activity] if i not in stereo_smiles_idx]
        activity_id = [act_id for i, act_id in enumerate(activity_id) if i not in stereo_smiles_idx]

        if len(stereo_smiles_idx) > 0:
            print(f'Removed {len(stereo_smiles_idx)} stereoisomers, and the idx are {stereo_smiles_idx}')

    spectral = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=RANDOM_SEED)
    clusters = spectral.fit(get_substructure_matrix(smiles)).labels_

    train_act_id, test_act_id = [], []
    for cluster in range(n_clusters):
        cluster_idx = np.where(clusters == cluster)[0]
        clust_train_idx, clust_test_idx = train_test_split(cluster_idx, test_size=test_size, random_state=RANDOM_SEED, shuffle=True)
        # get the corresponding 'activity_id'
        cluster_train_act_id = [activity_id[i] for i in clust_train_idx]
        cluster_test_act_id = [activity_id[i] for i in clust_test_idx]

        train_act_id.extend(cluster_train_act_id)
        test_act_id.extend(cluster_test_act_id)

    return train_act_id, test_act_id

"""Adopted from https://github.com/rinikerlab/molecular_time_series.git/"""
def clusterData(dmat, threshold, clusterSizeThreshold, combineRandom=False):
    """ 
    Cluster data based on a distance matrix using the Butina algorithm.

    Params
    ------
    - dmat: a distance matrix get from `get_substructure_matrix`, `get_scaffold_matrix`, or `get_levenshtein_matrix`
    - threshold: float, the distance threshold for clustering. E.g. 0.4 means mols with a similarity above 0.6 will be grouped into the same cluster.
    - clusterSizeThreshold: int, the minimum size for a cluster to be considered "large". Clusters smaller than this size will be handled according to the `combineRandom` parameter.
    - combineRandom: bool, if True, small clusters will be combined randomly to form larger clusters. If False, points from small clusters will be added to the nearest larger cluster based on the distance matrix.

    Returns
    -------
    - largeClusters: List of clusters, where each cluster is represented as a list of indices
    """
    nfps = len(dmat)
    symmDmat = []
    for i in range(1, nfps):
        symmDmat.extend(dmat[i, :i]) # convert a square distance matrix to a 1D array representing the upper triangle of the matrix
    cs = Butina.ClusterData(symmDmat, nfps, threshold, isDistData=True, reordering=True)
    cs = sorted(cs, key=lambda x: len(x), reverse=True) # sort clusters by size in descending order

    # start with the large clusters:
    largeClusters = [list(c) for c in cs if len(c) >= clusterSizeThreshold]
    if not largeClusters:
        raise ValueError("no clusters found")
    # now combine the small clusters to make larger ones:
    if combineRandom:
        tmpCluster = []
        for c in cs:
            if len(c) >= clusterSizeThreshold:
                continue
            tmpCluster.extend(c)
            if len(tmpCluster) >= clusterSizeThreshold:
                random.shuffle(tmpCluster)
                largeClusters.append(tmpCluster)
                tmpCluster = []
        if tmpCluster:
            largeClusters.append(tmpCluster)
    else:
        # add points from small clusters to the nearest larger cluster
        #  nearest is defined by the nearest neighbor in that cluster
        for c in cs:
            if len(c) >= clusterSizeThreshold:
                continue
            for idx in c:
                closest = -1
                minD = 1e5
                for cidx, clust in enumerate(largeClusters):
                    for elem in clust:
                        d = dmat[idx, elem]
                        if d < minD:
                            closest = cidx
                            minD = d
                assert closest > -1
                largeClusters[closest].append(idx)
    return largeClusters

def assignUsingClusters(dmat, threshold, clusterSizeThreshold=5, combineRandom=False, 
                        random_seed=RANDOM_SEED, test_size=0.2,n_folds=5, selectionStrategy='clust_holdout', ):
    """
    Assigns data points to training and testing sets based on selection strategies using clustering.

    Params
    ------
    - dmat: a distance matrix get from `get_substructure_matrix`, `get_scaffold_matrix`, or `get_levenshtein_matrix`
    - threshold: float, the distance threshold for clustering. Molecules with distances below this threshold will be grouped into the same cluster.
    - clusterSizeThreshold: int, the minimum size for a cluster to be considered "large". Clusters smaller than this size will be handled according to the `combineRandom` parameter.
    - combineRandom: bool, if True, small clusters will be combined randomly to form larger clusters. If False, points from small clusters will be added to the nearest larger cluster based on the distance matrix.
    
    - randomSeed: int, seed for random number generator to ensure reproducibility.
    - test_size: float, the proportion of the dataset to include in the test split.
    - n_folds: int, the number of different train-test splits to generate.
    - selectionStrategy: SelectionStrategy, the strategy to use for selecting test samples. Options are 'clust_stratified' and 'clust_holdout'.
    
    Returns
    -------
    - assignments: List[List[int]], a list containing n_folds lists, each representing the indices of the test samples.
    """

    largeClusters = clusterData(dmat, threshold, clusterSizeThreshold, combineRandom)
    
    random.seed(random_seed) # set the random seed for reproducibility
    nTest= round(len(dmat)*test_size)
    assignments = [] # list of lists, each containing the indices of the test samples for each split
    for i in range(n_folds): 
        # ensure distributional overlap between train and test splits -> easier task
        if selectionStrategy == 'clust_stratified': 
            ordered = []
            for c in largeClusters:
                random.shuffle(c) # shuffle the 
                ordered.extend((i / len(c), x) for i, x in enumerate(c))
            ordered = [y for x, y in sorted(ordered)]
            test=ordered[:nTest]
        # ensure cluster disjointness - train and test cover different regions of chemical space -> harder task
        elif selectionStrategy == 'clust_holdout': 
            random.shuffle(largeClusters)
            test = []
            for c in largeClusters:
                nRequired = nTest - len(test)
                test.extend(c[:nRequired])
                if len(test) >= nTest:
                    break
        assignments.append(test)

    return assignments

def cluster_aware_split(dist_type, selectionStrategy, x, 
                        threshold=0.65, combineRandom=False, 
                        random_seed=RANDOM_SEED, test_size=0.2, n_folds=5):
    """
    params
    ------
    - dist_type: str, the type of distance metric to use. Options are 'substruct', 'scaf', and 'levenshtein'.
    - selectionStrategy: SelectionStrategy, the strategy to use for selecting test samples. Options are 'clust_stratified' and 'clust_holdout'.
         according to my observation,
          - 'cluster_stratified' ensures each fold has different data points for all kinds of datasets (hhd, mhd, lhd, small or large)
          - 'cluster_holdout' only ensure each fold for mhd and hhd has different data points, but for lhd or very small datasets, some folds may have identical data points


    returns
    -----------
    - test_folds: List[np.ndarray]
        A list of lists, each containing the indices of the test set for each sampling.
    """
    if dist_type == 'substruct':
        dmat = get_substructure_dmat(x)
    elif dist_type == 'scaf':
        dmat = get_scaffold_dmat(x)
    elif dist_type == 'levenshtein':
        dmat = get_levenshtein_dmat(x)
    
    clusterSizeThreshold=max(5, len(x)/50) # set a minimum cluster size based on the dataset size

    test_folds = assignUsingClusters(dmat, threshold, clusterSizeThreshold, combineRandom,
                                     random_seed, test_size, n_folds, selectionStrategy)
    return test_folds

# Main

In [85]:
in_path = '/storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/data_prep/data_curate/old/cura_lhd_or'
#in_path = '/storage/homefs/yc24j783/datacat4ml/datacat4ml/Data/data_prep/data_curate/old/cura_mhd_or'
#in_file = 'CHEMBL236_bind_RBA_Ki_CHEMBL3887031_lhd_b50_curated.csv' # 185 points
in_file = 'CHEMBL237_bind_RBA_Ki_CHEMBL892113_lhd_b50_curated.csv' # 53 points
#in_file = 'CHEMBL237_bind_RBA_Ki_CHEMBL5253113_lhd_s50_curated.csv' # 7 points
#in_file = 'CHEMBL233_bind_RBA_Ki_mhd_b50_curated.csv' # 4472 points
df = pd.read_csv(os.path.join(in_path, in_file))
print(f'The shape of the data is {df.shape}')
df.columns

The shape of the data is (53, 49)


Index(['Unnamed: 0', 'activity_id', 'assay_id', 'assay_chembl_id', 'tid',
       'target_chembl_id', 'standard_type', 'standard_relation',
       'standard_value', 'standard_units', 'pchembl_value', 'assay_type',
       'assay_type_description', 'assay_category', 'assay_organism',
       'assay_tax_id', 'assay_strain', 'assay_tissue', 'assay_cell_type',
       'assay_subcellular_fraction', 'bao_format', 'bao_label', 'variant_id',
       'assay_test_type', 'assay_description', 'cell_id', 'tissue_id',
       'curated_by', 'aidx', 'confidence_score',
       'confidence_score_description', 'molregno', 'compound_chembl_id',
       'canonical_smiles', 'assay_metadata_hash', 'canonical_smiles_by_Std',
       'molecular_weight', 'num_atoms', 'pStandard_value',
       'vs_activity_comment', 'vs_activity', 'vs_threshold',
       'lo_activity_comment', 'lo_activity', 'lo_threshold', 'effect', 'assay',
       'effect_description', 'assay_keywords_description'],
      dtype='object')

In [86]:
df[['canonical_smiles_by_Std', 'lo_activity', 'activity_id']]

,canonical_smiles_by_Std,lo_activity,activity_id
0,OC1(c2ccccc2)CCN(C(c2ccccc2)c2ccccc2)CC1,1.0,1951530
1,CC(C)S(=O)(=O)N1CCN(Cc2ccccc2C2(O)CCN(C(c3cccc...,1.0,1951684
2,CCS(=O)(=O)N1CCN(Cc2ccccc2C2(O)CCN(C(c3ccccc3C...,1.0,1951685
3,CN1CCN(Cc2ccccc2C2(O)CCN(C(c3ccccc3Cl)c3ccccc3...,1.0,1951686
4,OC1(c2ccccc2CN2CCNCC2)CCN(C(c2ccccc2Cl)c2ccccc...,1.0,1951687
5,OC1(c2ccccc2CN2CCOCC2)CCN(C(c2ccccc2Cl)c2ccccc...,1.0,1951688
6,CCNC(=O)NCc1ccccc1C1(O)CCN(C(c2ccccc2Cl)c2cccc...,1.0,1951689
7,CNC(=O)NCc1ccccc1C1(O)CCN(C(c2ccccc2Cl)c2ccccc...,1.0,1951690
8,CC(C)S(=O)(=O)NCc1ccccc1C1(O)CCN(C(c2ccccc2Cl)...,1.0,1951691
9,CCCS(=O)(=O)NCc1ccccc1C1(O)CCN(C(c2ccccc2Cl)c2...,1.0,1951692


In [87]:
smiles = df['canonical_smiles_by_Std'].tolist()[:]
activity = df['lo_activity'].tolist()[:]
activity_id = df['activity_id'].tolist()[:]
print(f'Number of data points: {len(smiles)}')

Number of data points: 53


## random_split

In [79]:
# random split
test_folds = random_split(smiles, activity, n_folds=5, test_size=0.2, random_seed=RANDOM_SEED)
test_folds

[[3039,
  3031,
  1721,
  1064,
  3122,
  3392,
  1991,
  4024,
  1944,
  1605,
  2660,
  190,
  856,
  2052,
  1465,
  2971,
  84,
  3879,
  2887,
  2584,
  4148,
  1935,
  3183,
  4365,
  3206,
  1236,
  3899,
  2316,
  1494,
  436,
  2066,
  2539,
  3551,
  2412,
  1844,
  656,
  1153,
  2000,
  1444,
  2419,
  2314,
  2804,
  3300,
  3584,
  4270,
  1842,
  2991,
  876,
  3251,
  3841,
  4258,
  4385,
  149,
  279,
  978,
  1867,
  3293,
  1642,
  3470,
  1431,
  2988,
  3542,
  4249,
  1145,
  510,
  2702,
  1206,
  289,
  1428,
  1479,
  2940,
  2654,
  2499,
  3971,
  2181,
  3666,
  2681,
  3731,
  1065,
  4065,
  3783,
  68,
  929,
  277,
  602,
  2458,
  13,
  1530,
  1441,
  2276,
  2337,
  1731,
  1769,
  2024,
  3401,
  3608,
  3437,
  1535,
  449,
  3491,
  3351,
  3451,
  310,
  1446,
  3746,
  2158,
  914,
  3571,
  4105,
  2620,
  1940,
  1414,
  1553,
  3751,
  50,
  2974,
  4398,
  4123,
  2919,
  3041,
  3787,
  3648,
  3181,
  2003,
  3439,
  2275,
  107,
  1497,
 

In [93]:
#cluster_aware_split: clust_stratified
test_folds_2 = cluster_aware_split(dist_type= 'substruct', selectionStrategy='clust_stratified', x=smiles,
                                   threshold=0.65, combineRandom=False, random_seed=RANDOM_SEED, test_size=0.2, n_folds=5)
## check whether there are identical elements in 'test_folds_3'
tupled_folds_2 = [tuple(sorted(fold)) for fold in test_folds_2]
has_duplicates = len(tupled_folds_2) != len(set(tupled_folds_2))
print(f'has_duplicates in test_folds_2: {has_duplicates}')
test_folds_2

has_duplicates in test_folds_2: False


[[8, 22, 24, 2, 20, 37, 15, 39, 18, 10, 49],
 [3, 10, 22, 16, 11, 44, 46, 28, 8, 34, 2],
 [15, 28, 24, 48, 39, 25, 46, 13, 23, 45, 2],
 [40, 22, 25, 4, 19, 11, 44, 50, 35, 21, 17],
 [29, 6, 8, 13, 15, 21, 23, 27, 24, 32, 7]]

In [ ]:
# cluster_stratified_split
test_folds_3 = cluster_aware_split(dist_type= 'substruct', selectionStrategy='clust_holdout', x=smiles, 
                                   threshold=0.65, combineRandom=False, random_seed=RANDOM_SEED, test_size=0.2, n_folds=5)
test_folds_3
## check whether there are identical elements in 'test_folds_3'
tupled_folds_3 = [tuple(fold) for fold in test_folds_3]
has_duplicates = len(tupled_folds_3) != len(set(tupled_folds_3))


{(384,
  36,
  37,
  46,
  66,
  69,
  75,
  81,
  91,
  100,
  104,
  108,
  109,
  110,
  113,
  114,
  115,
  116,
  119,
  120,
  123,
  124,
  126,
  127,
  128,
  132,
  133,
  136,
  137,
  140,
  143,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  196,
  226,
  227,
  228,
  230,
  233,
  236,
  237,
  240,
  243,
  246,
  247,
  248,
  249,
  250,
  252,
  253,
  254,
  255,
  256,
  258,
  259,
  260,
  261,
  262,
  264,
  299,
  300,
  302,
  304,
  305,
  306,
  307,
  308,
  309,
  310,
  311,
  312,
  313,
  314,
  315,
  316,
  317,
  319,
  382,
  383,
  385,
  411,
  415,
  421,
  430,
  437,
  444,
  451,
  453,
  456,
  463,
  469,
  471,
  474,
  535,
  681,
  682,
  683,
  684,
  685,
  834,
  835,
  836,
  837,
  838,
  839,
  840,
  841,
  842,
  844,
  861,
  862,
  863,
  864,
  865,
  866,
  867,
  868,
  869,
  870,
  871,
  872,
  873,
  874,
  875,
  876,
  877,
  936,
  937,
  938,


In [82]:
has_duplicates

False

## Utility functions for splitting

In [ ]:
def read_df(in_path: str, file: str, aim: str = 'lo', remove_stereo: bool = True):
    """
    """
    df = pd.read_csv(os.path.join(in_path, file))

    smiles = df['canonical_smiles_by_Std'].tolist()
    if aim == 'vs':
        activity = df['vs_activity'].tolist()
    elif aim == 'lo':
        activity = df['lo_activity'].tolist()
    activity_id = df['activity_id'].tolist()

    if remove_stereo:
        stereo_smiles = find_stereochemical_siblings(smiles)
        stereo_smiles_idx = [i for i, smi in enumerate(smiles) if smi in stereo_smiles]

        if len(stereo_smiles_idx) == 0:
            print('No stereoisomers found')
        else:
            print(f'Removed {len(stereo_smiles_idx)} stereoisomers, and the idx are {stereo_smiles_idx}')
            smiles = [smi for i, smi in enumerate(smiles) if i not in stereo_smiles_idx]
            activity = [act for i, act in enumerate(activity) if i not in stereo_smiles_idx]
            activity_id = [act_id for i, act_id in enumerate(activity_id) if i not in stereo_smiles_idx]
            
            df = df.drop(index=stereo_smiles_idx).reset_index(drop=True)

    return df, smiles, activity, activity_id

def assign_split(n_folds: int = 5):
    
    sub_sups_dict = {
        CURA_LHD_OR_DIR: (CURA_MHD_OR_DIR,CURA_MHD_OR_effect_DIR, CURA_HHD_OR_DIR),
        #CURA_MHD_OR_DIR: (CURA_MHD_OR_effect_DIR, CURA_HHD_OR_DIR),
    }

    cura_split_dict = {
        CURA_LHD_OR_DIR: SPLIT_LHD_OR_DIR,
        CURA_MHD_OR_DIR: SPLIT_MHD_OR_DIR,
        CURA_MHD_OR_effect_DIR: SPLIT_MHD_OR_effect_DIR,
        CURA_HHD_OR_DIR: SPLIT_HHD_OR_DIR,
    }

    for sub, sups in sub_sups_dict.items():
        #print(f'Subset: {sub}')
        for sub_f in os.listdir(sub):
            print(f'sub_f is {sub_f}')
            if sub == CURA_LHD_OR_DIR:
                target, assay, effect, standard_type, assay_chembl_id = sub_f.split('_')[:5]
            elif sub == CURA_MHD_OR_DIR:
                target, assay, effect, standard_type = sub_f.split('_')[:4]

            sub_df, smiles, activity, activity_id = read_df(sub, sub_f, aim='lo', remove_stereo=True)
            fold_train_act_id, fold_test_act_id = random_split(smiles, activity, activity_id, n_folds=5, test_size=0.2)

            for sup in sups:
                if sup == CURA_MHD_OR_DIR:
                    sup_base_name = f'{target}_{assay}_{effect}_{standard_type}'
                elif sup == CURA_MHD_OR_effect_DIR:
                    sup_base_name = f'{target}_{effect}'
                elif sup == CURA_HHD_OR_DIR:
                    sup_base_name = f'{target}_{standard_type}'
                
                sup_fs = [f for f in os.listdir(sup) if f.startswith(sup_base_name)]
                print (f'sup_fs found: {sup_fs}')
                if len(sup_fs) == 1:
                    sup_f = sup_fs[0]
                elif len(sup_fs) == 0:
                    print(f'No sup_f found for {sup_base_name} in {sup}')
                    continue
                else:
                    print(f'More than one sup_f found for {sup_base_name} in {sup}, please check!')
                    continue
                print(f'sup_f found: {sup_f}')
                sup_df, _, _, _ = read_df(sup, sup_f, aim='lo', remove_stereo=True)

                for i in range(n_folds):
                    sub_df[f'fold{i}'] = ['test' if act_id in fold_test_act_id[f'fold_{i}'] else 'train' for act_id in sub_df['activity_id']]
                    sup_df[f'{sub_f}:fold{i}'] = ['test' if act_id in fold_test_act_id[f'fold_{i}'] else 'train' for act_id in sup_df['activity_id']]

                mkdirs(cura_split_dict[sub])
                mkdirs(cura_split_dict[sup])
                sub_df.to_csv(os.path.join(cura_split_dict[sub], sub_f), index=False)
                sup_df.to_csv(os.path.join(cura_split_dict[sup], sup_f), index=False)

            print('----------------------------------')
assign_split()

sub_f is CHEMBL233_bind_RBA_Ki_CHEMBL3887789_lhd_b50_curated.csv


100%|██████████| 90/90 [00:00<00:00, 11648.69it/s]

Removed 2 stereoisomers, and the idx are [32, 33]
sup_fs found: ['CHEMBL233_bind_RBA_Ki_mhd_b50_curated.csv']
sup_f found: CHEMBL233_bind_RBA_Ki_mhd_b50_curated.csv



100%|██████████| 4472/4472 [00:19<00:00, 233.32it/s] 


Removed 633 stereoisomers, and the idx are [1, 3, 6, 7, 10, 11, 12, 13, 14, 15, 16, 20, 21, 22, 25, 31, 33, 34, 35, 37, 39, 40, 43, 45, 46, 51, 53, 59, 66, 76, 79, 80, 87, 88, 92, 97, 99, 100, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 186, 188, 192, 194, 196, 202, 204, 207, 212, 214, 216, 218, 223, 227, 228, 229, 232, 233, 239, 240, 241, 244, 257, 268, 269, 278, 283, 291, 293, 348, 351, 387, 388, 389, 391, 394, 397, 398, 399, 401, 405, 412, 414, 416, 434, 438, 441, 445, 452, 457, 459, 462, 465, 470, 475, 479, 481, 482, 483, 493, 498, 501, 522, 523, 524, 526, 527, 528, 529, 530, 531, 532, 533, 534, 536, 537, 538, 539, 540, 541, 542, 545, 546, 547, 548, 549, 612, 645, 647, 655, 656, 668, 703, 731, 739, 740, 744, 745, 746, 814, 847, 850, 852, 853, 854, 885, 886, 889, 890, 891, 927, 928, 929, 930, 931, 932, 933, 934, 935, 952, 1019, 1020, 1021, 1022, 1023, 1024, 1028, 1029, 1030, 1031, 1033, 1035, 1037, 1038, 1039, 1040, 1

100%|██████████| 4535/4535 [00:19<00:00, 232.91it/s]


Removed 647 stereoisomers, and the idx are [1, 3, 6, 7, 10, 11, 12, 13, 14, 15, 16, 20, 21, 22, 25, 31, 33, 34, 35, 37, 39, 40, 43, 45, 46, 51, 53, 59, 66, 76, 79, 80, 87, 88, 92, 97, 99, 100, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 178, 187, 189, 193, 195, 197, 203, 205, 208, 213, 215, 217, 219, 224, 228, 229, 230, 233, 234, 240, 241, 242, 245, 259, 270, 271, 280, 285, 293, 295, 359, 362, 424, 425, 426, 428, 431, 434, 435, 436, 438, 442, 449, 451, 453, 471, 475, 478, 482, 489, 496, 498, 502, 507, 515, 523, 527, 529, 530, 531, 541, 546, 549, 570, 571, 572, 574, 575, 576, 577, 578, 579, 580, 581, 582, 584, 585, 586, 587, 589, 590, 591, 594, 595, 596, 597, 598, 661, 692, 694, 702, 703, 715, 744, 745, 751, 752, 753, 754, 755, 756, 767, 795, 803, 804, 808, 809, 810, 878, 911, 914, 916, 917, 918, 949, 950, 953, 954, 955, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1016, 1083, 1084, 1085, 1086, 1087, 1088, 1092, 1093, 109

100%|██████████| 68/68 [00:00<00:00, 15592.21it/s]

No stereoisomers found


ValueError: The target variable 'activity' must contain at least two classes for stratified splitting.

In [ ]:
class CrossValidationOrchestrator:
    def __init__(self, data_dict: Dict[str, pd.DataFrame]):
        """
        params:
        ---------
        - data_dict: Dictionary mapping category ('HHD', 'MHD', etc.) to its DataFrame.
        """
        self.data = data_dict
        self.comparison_paris = [
            ('LHD', 'MHD'), ('LHD', 'MHD_effect'), ('LHD', 'HHD'),
            ('MHD', 'MHD_effect'), ('MHD', 'HHD'),
            ('MHD_effect', 'HHD')
        ]

        def _get_split_indices(self, df_subset: pd.DataFrame, split_method: str):
            """
            """
            if split_method == 'random':
                fold_train_act_id, fold_test_act_id = random_split(

In [ ]:
for i in range(n_folds):
    print(f'Fold {i}:')
    print(f'Train activity IDs: {lhd_train_act_id[f"fold_{i}"]}')
    print(f'Test activity IDs: {lhd_test_act_id[f"fold_{i}"]}')
    print('----------------------------------')


Fold 0:
Train activity IDs: [1951687, 1951718, 1951705, 1951693, 1951684, 1951688, 1951716, 1951700, 1951713, 1951736, 1951690, 1951694, 1951708, 1951721, 1951696, 1951738, 1951740, 1951685, 1951725, 1951734, 1951717, 1951703, 1951530, 1951691, 1951712, 1951739, 1951697, 1951707, 1951737, 1951704, 1951720, 1951686, 1951698, 1951733, 1951710, 1951706, 1951689, 1951728, 1951727, 1951702, 1951729, 1951722]
Test activity IDs: [1951714, 1951715, 1951735, 1951731, 1951719, 1951724, 1951692, 1951695, 1951699, 1951701, 1951732]
----------------------------------
Fold 1:
Train activity IDs: [1951705, 1951737, 1951701, 1951703, 1951728, 1951720, 1951729, 1951700, 1951717, 1951732, 1951715, 1951739, 1951716, 1951702, 1951724, 1951731, 1951691, 1951713, 1951692, 1951704, 1951719, 1951727, 1951722, 1951735, 1951693, 1951725, 1951689, 1951699, 1951696, 1951684, 1951707, 1951686, 1951736, 1951708, 1951733, 1951695, 1951698, 1951721, 1951710, 1951690, 1951688, 1951734]
Test activity IDs: [1951694, 195

## neighbor_split

In [ ]:
# def clusterData
dmat = get_substructure_matrix(smiles)
dmat

100%|██████████| 53/53 [00:00<00:00, 19428.26it/s]


array([[0.        , 0.30645161, 0.31666667, ..., 0.79411765, 0.75      ,
        0.64864865],
       [0.30645161, 0.        , 0.8245614 , ..., 0.28358209, 0.29411765,
        0.34375   ],
       [0.31666667, 0.8245614 , 0.        , ..., 0.29230769, 0.3030303 ,
        0.35483871],
       ...,
       [0.79411765, 0.28358209, 0.29230769, ..., 0.        , 0.74358974,
        0.65      ],
       [0.75      , 0.29411765, 0.3030303 , ..., 0.74358974, 0.        ,
        0.61904762],
       [0.64864865, 0.34375   , 0.35483871, ..., 0.65      , 0.61904762,
        0.        ]])

In [ ]:
threshold = 0.3
#clusterSizeThreshold = max(5, len(dmat)/50)
clusterSizeThreshold = 2
print(f'Cluster size threshold is set to {clusterSizeThreshold}')
combineRandom=False

nfps = len(smiles)
print(f'Number of fingerprints: {nfps}')
symmDmat = []
for i in range(1, nfps):
    symmDmat.extend(dmat[i, :i]) # the list of values in the lower triangle of the distance matrix, excluding the diagonal.
print(f'The length of symmDmat is {len(symmDmat)}')
symmDmat

Cluster size threshold is set to 2
Number of fingerprints: 53
The length of symmDmat is 1378


[0.3064516129032258,
 0.31666666666666665,
 0.8245614035087719,
 0.34545454545454546,
 0.7368421052631579,
 0.7636363636363637,
 0.34545454545454546,
 0.6779661016949152,
 0.7017543859649122,
 0.7692307692307693,
 0.34545454545454546,
 0.6779661016949152,
 0.7017543859649122,
 0.7692307692307693,
 0.7692307692307693,
 0.3275862068965517,
 0.59375,
 0.639344262295082,
 0.6379310344827587,
 0.6101694915254238,
 0.6101694915254238,
 0.3333333333333333,
 0.6031746031746031,
 0.6229508196721312,
 0.6491228070175439,
 0.6206896551724138,
 0.6206896551724138,
 0.8301886792452831,
 0.3220338983050847,
 0.6885245901639344,
 0.6557377049180327,
 0.6271186440677966,
 0.6,
 0.6,
 0.6779661016949152,
 0.6896551724137931,
 0.31666666666666665,
 0.6507936507936508,
 0.7,
 0.6440677966101694,
 0.5901639344262295,
 0.5901639344262295,
 0.6949152542372882,
 0.6779661016949152,
 0.7719298245614035,
 0.36538461538461536,
 0.6271186440677966,
 0.6491228070175439,
 0.7115384615384616,
 0.6792452830188679,
 

In [ ]:
cs = Butina.ClusterData(symmDmat, nfps, threshold, isDistData=True, reordering=True)
cs

((11,
  0,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  39,
  40,
  41,
  43,
  44,
  45,
  46,
  48,
  49,
  50,
  51,
  52),
 (42, 12),
 (47,),
 (38,),
 (25,),
 (24,),
 (23,),
 (22,),
 (21,),
 (20,),
 (19,),
 (18,),
 (17,),
 (16,),
 (15,),
 (14,),
 (13,),
 (10,),
 (9,),
 (8,),
 (7,),
 (6,),
 (5,),
 (4,),
 (3,),
 (2,),
 (1,))

In [ ]:
cs = sorted(cs, key=lambda x: len(x), reverse=True)
cs

[(11,
  0,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  39,
  40,
  41,
  43,
  44,
  45,
  46,
  48,
  49,
  50,
  51,
  52),
 (42, 12),
 (47,),
 (38,),
 (25,),
 (24,),
 (23,),
 (22,),
 (21,),
 (20,),
 (19,),
 (18,),
 (17,),
 (16,),
 (15,),
 (14,),
 (13,),
 (10,),
 (9,),
 (8,),
 (7,),
 (6,),
 (5,),
 (4,),
 (3,),
 (2,),
 (1,)]

In [ ]:
# start with the large clusters
largeClusters = [list(c) for c in cs if len(c) >= clusterSizeThreshold]
largeClusters

[[11,
  0,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  39,
  40,
  41,
  43,
  44,
  45,
  46,
  48,
  49,
  50,
  51,
  52],
 [42, 12]]

In [ ]:
if not largeClusters:
    raise ValueError("no clusters found")
if combineRandom:
    tmpCluster = []
    for c in cs:
        if len(c) >= clusterSizeThreshold:
            continue
        tmpCluster.extend(c)
        if len(tmpCluster) >= clusterSizeThreshold:
            random.shuffle(tmpCluster)
            largeClusters.append(tmpCluster)
            tmpCluster = []
    if tmpCluster:
        largeClusters.append(tmpCluster)
else:
    # add points from small cluster to the nearest large cluster
    # nearest is defined by the nearest neighbor in that cluster
    print(f'cs is {cs}')
    for c in cs:
        print(f'c: {c}')
        if len(c) >= clusterSizeThreshold:
            continue
        for idx in c:
            print(f'idx: {idx}')
            closest = -1
            minD = 1e5
            print(f'minD is {minD}')
            for cidx, clust in enumerate(largeClusters):
                print(f'cidx: {cidx}, clust: {clust}')
                for elem in clust:
                    print(f'idx, elem: {idx}, {elem}')
                    d = dmat[idx, elem]
                    print(f'd: {d}')
                    if d < minD:
                        closest = cidx
                        minD = d
            assert closest > -1
            print(f'closest is {closest}')
            print(f'minD is {minD}')
            largeClusters[closest].append(idx)
            print(f'largeClusters is now {largeClusters}')
            print('=====================')

print(f'largeClusters: {largeClusters}')

cs is [(11, 0, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 48, 49, 50, 51, 52), (42, 12), (47,), (38,), (25,), (24,), (23,), (22,), (21,), (20,), (19,), (18,), (17,), (16,), (15,), (14,), (13,), (10,), (9,), (8,), (7,), (6,), (5,), (4,), (3,), (2,), (1,)]
c: (11, 0, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 48, 49, 50, 51, 52)
c: (42, 12)
c: (47,)
idx: 47
minD is 100000.0
cidx: 0, clust: [11, 0, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 48, 49, 50, 51, 52]
idx, elem: 47, 11
d: 0.32142857142857145
idx, elem: 47, 0
d: 0.6486486486486487
idx, elem: 47, 26
d: 0.5714285714285714
idx, elem: 47, 27
d: 0.627906976744186
idx, elem: 47, 28
d: 0.5714285714285714
idx, elem: 47, 29
d: 0.5581395348837209
idx, elem: 47, 30
d: 0.4782608695652174
idx, elem: 47, 31
d: 0.5238095238095238
idx, elem: 47, 32
d: 0.5
idx, elem: 47, 33
d: 0.5
idx, elem: 47, 34
d: 0.5897435897435898
idx, elem: 47, 35
d: 0.5227272

In [ ]:
# def assignUsingClusters
nSamples=4

class SelectionStrategy:
    DIVERSE_SPLIT = 1
    CLUSTERS_SPLIT = 2
selectionStrategy = SelectionStrategy.CLUSTERS_SPLIT
print(f'Selection strategy is {selectionStrategy}')
test_size = 0.2
nTest= round(len(smiles)*test_size)
print(f'Number of test samples: {nTest}')

random.seed(RANDOM_SEED)
res = []
for i in range(nSamples):
    print(f'Iteration {i}')
    if selectionStrategy == SelectionStrategy.DIVERSE_SPLIT: 
    # The test set has the similar distribution of clusters as the training set
        ordered = []
        for c in largeClusters:
            print(f'c before shuffle: {c}')
            random.shuffle(c)
            print(f'c after shuffle: {c}')
            ordered.extend((i / len(c), x) for i, x in enumerate(c))
            print(f'ordered in: {ordered}')
            print('-------------------')

        print(f'ordered before sort: {ordered}')
        ordered = [y for x, y in sorted(ordered)]
        print(f'ordered after sort: {ordered}')
        test=ordered[:nTest]
        print(f'test: {test}')
    elif selectionStrategy == SelectionStrategy.CLUSTERS_SPLIT:
        # The test set has the different distribution of clusters as the training set
        random.shuffle(largeClusters)
        print(f'largeClusters after shuffle: {largeClusters}')
        test = []
        for c in largeClusters:
            nRequired = nTest - len(test)
            print(f'nRequired: {nRequired}, len(test): {len(test)}')
            test.extend(c[:nRequired])
            if len(test) >= nTest:
                break
        print(f'test: {test}')
    
    res.append(test)
    print(f'res: {res}')
    print('----------------------------------')

print(f'Final result: {res}')

Selection strategy is 2
Number of test samples: 11
Iteration 0
largeClusters after shuffle: [[18, 21, 51, 29, 28, 32, 13, 0, 19, 50, 31, 5, 36, 10, 22, 8, 2, 48, 14, 9, 24, 40, 45, 16, 44, 37, 15, 38, 26, 6, 27, 33, 20, 11, 1, 25, 41, 43, 30, 4, 7, 17, 39, 49, 52, 23, 35, 34, 3, 46], [42, 12, 47]]
nRequired: 11, len(test): 0
test: [18, 21, 51, 29, 28, 32, 13, 0, 19, 50, 31]
res: [[18, 21, 51, 29, 28, 32, 13, 0, 19, 50, 31]]
----------------------------------
Iteration 1
largeClusters after shuffle: [[42, 12, 47], [18, 21, 51, 29, 28, 32, 13, 0, 19, 50, 31, 5, 36, 10, 22, 8, 2, 48, 14, 9, 24, 40, 45, 16, 44, 37, 15, 38, 26, 6, 27, 33, 20, 11, 1, 25, 41, 43, 30, 4, 7, 17, 39, 49, 52, 23, 35, 34, 3, 46]]
nRequired: 11, len(test): 0
nRequired: 8, len(test): 3
test: [42, 12, 47, 18, 21, 51, 29, 28, 32, 13, 0]
res: [[18, 21, 51, 29, 28, 32, 13, 0, 19, 50, 31], [42, 12, 47, 18, 21, 51, 29, 28, 32, 13, 0]]
----------------------------------
Iteration 2
largeClusters after shuffle: [[42, 12, 47

In [ ]:
# Random split
# neighbor split: train_test_diff
#                 train_test_similar
# inner_split: lhd_or, mhd_or, mhd_or_effect
# outer_split: 
#               based on lhd_or: mhd_or, mhd_or_effect, hhd_or
#               based on mhd_or: mhd_or_effect, hhd_or
#               based on mhd_or_effect: hhd_or

# cross-validation for hyperparameter tuning
# nested cross-validation for model evaluation


# assay_wise_split on MHDs and HHDs